# Appendix B

## 1. *Generalization* 
Install Keras. Go to the Keras MNIST example. Perform a classification task. Note how many epochs the training takes, and in testing, how well it generalizes. Perform the classification on a smaller training set, how does learning rate change, how does generalization change. Vary other elements: try a different optimizer than adam, try a different learning rate, try a different (deeper) architecture, try wider hidden layers. Does it learn faster? Does it generalize better?


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers

In [11]:
# https://keras.io/examples/vision/mnist_convnet/
num_classes = 18
input_shape = (28, 28, 1)

# the data, split between train and test sets
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
X_train = X_train.astype("float32") / 255
X_test = X_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)
print("x_train shape:", X_train.shape)
print(X_train.shape[0], "train samples")
print(X_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [12]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 


2022-04-19 16:39:00.165471: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-19 16:39:00.165607: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-19 16:39:00.165666: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ee1eba175434): /proc/driver/nvidia/version does not exist
2022-04-19 16:39:00.166342: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 18)                28818     
                                                                 
Total params: 47,634
Trainable params: 47,634
Non-trainable params: 0
_________________________________________________________________


In [13]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

2022-04-19 16:40:06.917286: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 169344000 exceeds 10% of free system memory.


Epoch 1/15
422/422 [==============================] - 84s 195ms/step - loss: 0.4033 - accuracy: 0.8752 - val_loss: 0.0942 - val_accuracy: 0.9750
Epoch 2/15
422/422 [==============================] - 79s 188ms/step - loss: 0.1285 - accuracy: 0.9616 - val_loss: 0.0712 - val_accuracy: 0.9818
Epoch 3/15
422/422 [==============================] - 86s 205ms/step - loss: 0.0916 - accuracy: 0.9721 - val_loss: 0.0500 - val_accuracy: 0.9875
Epoch 4/15
422/422 [==============================] - 84s 200ms/step - loss: 0.0751 - accuracy: 0.9770 - val_loss: 0.0414 - val_accuracy: 0.9887
Epoch 5/15
422/422 [==============================] - 80s 190ms/step - loss: 0.0687 - accuracy: 0.9787 - val_loss: 0.0414 - val_accuracy: 0.9893
Epoch 6/15
422/422 [==============================] - 98s 233ms/step - loss: 0.0588 - accuracy: 0.9816 - val_loss: 0.0370 - val_accuracy: 0.9903
Epoch 7/15
422/422 [==============================] - 85s 202ms/step - loss: 0.0556 - accuracy: 0.9821 - val_loss: 0.0375 - val_ac

In [14]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.02899838238954544
Test accuracy: 0.989799976348877


## 2. *Overfitting* 
Use Keras again, but this time on ImageNet. Now try different over-fitting solutions. Does the training speed change? Does generalization change? Now try the hold-out validation set. Do training and generalization change?

## 3. *Confidence* 
How many runs did you do in the previous exercises, just a single run to see how long training took and how well generalization worked? Try to run it again. Do you get the same results? How large is the difference? Can you change the random seeds of Keras or TensorFlow? Can you calculate the confidence interval, how much does the confidence improve when you do 10 randomized runs? How about 100 runs? Make graphs with error bars.

## 4. *GPU*
It might be that you have access to a GPU machine that is capable of running PyTorch or TensorFlow in parallel to speed up the training. Install the GPU version and check that it recognizes the GPU and is indeed using it.

## 5. *Parallelism*
It might be that you have access to a multicore CPU machine. When you are running multiple runs in order to improve confiedence, then an easy way to speed up your experiment is to spawn multiple jobs at the shell, assigning the output to different log files, and write a script to combine results and draw graphs. Write the scripts necessary to achieve this, test them, and do a large-confidence experiment.